In [1]:
import pandas as pd
import numpy as np

import networkx as nx
from networkx.algorithms.community import k_clique_communities
import random as rd
rd.seed(42)

import warnings
warnings.filterwarnings('ignore')

## Random Walk Sampling Method 2
### Sample 6

In [2]:
G_rwm2_6 = nx.read_gml(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\Samples\Random Walk Sampling\G_rw2_6.gml')

In [3]:
# Confirm that Graph is Directed
nx.is_directed(G_rwm2_6)

True

In [4]:
# Display Size of the Graph
print(nx.info(G_rwm2_6))

MultiDiGraph with 50000 nodes and 139624 edges


Confirmed that the graph meets the criterea of having more edges than nodes and maintained the directed structure.

In [5]:
# List all nodes in the sample
n_rwm2_6 = list(G_rwm2_6.nodes)

In [6]:
# Get node properties from data exploration exercise for comparison
node_properties = pd.read_csv(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\node_properties_cl.csv')
node_properties[['degree', 'in_degree', 'out_degree', 'eigen_centrality']].describe()

,degree,in_degree,out_degree,eigen_centrality
count,4.666873e+06,4.666873e+06,4.666873e+06,4.666873e+06
mean,4.494298e+00,2.247149e+00,2.247149e+00,3.970074e-05
std,3.935922e+01,1.998412e+01,2.868249e+01,4.611943e-04
min,1.000000e+00,0.000000e+00,0.000000e+00,1.193322e-21
25%,1.000000e+00,0.000000e+00,0.000000e+00,1.193322e-21
50%,2.000000e+00,1.000000e+00,1.000000e+00,4.892620e-20
75%,3.000000e+00,1.000000e+00,2.000000e+00,1.517475e-14
max,2.791700e+04,1.211600e+04,2.533500e+04,1.642653e-01


In [7]:
rwm2_6_df = nx.to_pandas_edgelist(G_rwm2_6) 
rwm2_6_df.head()

,source,target,weight
0,3GbCwj8LfE1TnyDvC6kXL5RRaet9U5ZRoy,1Asn1M7ybhudPWkbJNPGH49dtGMM1GVvRi,1.0
1,3GbCwj8LfE1TnyDvC6kXL5RRaet9U5ZRoy,3DaQPZwsMszNJA4tsrvdpHie3WpuBST83y,1.0
2,1FLgTFrWYi2oKGSpRvrtk7yNArmMg1eU6k,1EkSWVk4nmzbiQ16qTDcbtDwz7JHfhSxQo,1.0
3,1NS35ayQpn39sYhiET3DsNXBL77MKs9yX,1J1Kf9dbLBGLB7aBrAF3F6ggdNEsXkGRcn,1.0
4,1NS35ayQpn39sYhiET3DsNXBL77MKs9yX,1PJSAbUV74eZXEj1PaZZqrDdbjTthBjNTz,1.0


In [8]:
# Number of unique input addresses
rwm2_6_df['source'].nunique()

31999

In [9]:
# Number of unique output addresses
rwm2_6_df['target'].nunique()

31378

We observe relative balance between input and output addresses, with slightly more input addresses. In the full dataset there is also relative balance, but there were more output addresses. 

In [10]:
rwm2_6_df.describe()

,weight
count,139624.000000
mean,0.767612
std,0.577350
min,-99.311045
25%,0.466874
50%,1.000000
75%,1.000000
max,52.821905


The weight variable shows that the distribution of the weighting (amount*0.5 + timestamp*0.5) is similar to that of overall transaction amounts.

In [11]:
# Subset by fraudulent/high-risk nodes
fr_nodes = node_properties[node_properties['fraud_flag'] == 1]
fr_nodes = fr_nodes['address'].tolist()
print('Total number of fraudulent/high-risk nodes : ', len(fr_nodes))

# Check for number of fraudulent/high-risk addresses in the sample
print('Number of fraudulent/high-risk nodes in sample : ', len(set(n_rwm2_6).intersection(fr_nodes)))

Total number of fraudulent/high-risk nodes :  4563
Number of fraudulent/high-risk nodes in sample :  301


These numbers suggest that random selection of nodes worsens the class imbalance problem, but when considering the counts relative to the size of the network the issue is less pronounced.

Full Network: 0.11% 

Sample: 0.60%

## Node Properties

In [12]:
# Get Properties

address = [node for (node, val) in G_rwm2_6.degree()]
degree = [val for (node, val) in G_rwm2_6.degree()]
in_degree = [val for (node, val) in G_rwm2_6.in_degree()]
out_degree = [val for (node, val) in G_rwm2_6.out_degree()]
#eigen_centrality = nx.eigenvector_centrality(G_rwm2_6) # NetworkXNotImplemented: not implemented for multigraph type
#eigen_centrality = [eigen_centrality[node] for node in eigen_centrality]

In [13]:
# Add Properties to DataFrame

nodes = pd.DataFrame()
nodes['address'] = address
nodes['degree'] = degree
nodes['in_degree'] = in_degree
nodes['out_degree'] = out_degree
#nodes['eigen_centrality'] = eigen_centrality

In [14]:
# Add fraud flag

nodes = nodes.assign(**dict.fromkeys(['fraud_flag'], 0))

for i in fr_nodes:
    nodes.loc[nodes.address == i, 'fraud_flag'] = 1
    
nodes.head()

,address,degree,in_degree,out_degree,fraud_flag
0,3GbCwj8LfE1TnyDvC6kXL5RRaet9U5ZRoy,2,0,2,0
1,1E4rXxSi8omoS8WzbAtDLfAzDgZnbNb3Tr,2,2,0,0
2,1FeqRvJcxxNs157qMTJX2HURQ6mDNy2ocG,1,1,0,0
3,1FLgTFrWYi2oKGSpRvrtk7yNArmMg1eU6k,1,0,1,0
4,1NS35ayQpn39sYhiET3DsNXBL77MKs9yX,5,3,2,0


In [15]:
# Describe sample properties overall

nodes[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,50000.000000,50000.000000,50000.000000
mean,5.584960,2.792480,2.792480
std,40.879206,18.119866,26.481189
min,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,2.000000,1.000000,1.000000
75%,4.000000,2.000000,2.000000
max,4434.000000,1648.000000,2852.000000


In [16]:
# Describe Properties of Fraudulent Nodes

nodes_fraud = nodes[nodes['fraud_flag'] == 1]
nodes_fraud[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,301.000000,301.000000,301.000000
mean,137.102990,57.026578,80.076412
std,475.856763,192.743091,311.453674
min,1.000000,0.000000,0.000000
25%,7.000000,2.000000,1.000000
50%,26.000000,10.000000,7.000000
75%,82.000000,39.000000,34.000000
max,4434.000000,1648.000000,2852.000000


In [17]:
# Describe Properties of Non-Fraudulent Nodes

nodes_licit = nodes[nodes['fraud_flag'] == 0]
nodes_licit[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,49699.000000,49699.000000,49699.000000
mean,4.788426,2.464013,2.324413
std,14.454810,9.388390,9.140545
min,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,2.000000,1.000000,1.000000
75%,4.000000,2.000000,2.000000
max,879.000000,876.000000,683.000000


For all 3 cases the degree distribution properties are similar to the overall network.

## Graph Properties

In [18]:
# Density
nx.density(G_rwm2_6) # Connectivity remains very low

5.5850717014340285e-05

In [19]:
# Communities - k-Cliques

com_generator_rwm2_6 = k_clique_communities(G_rwm2_6.to_undirected(), k=10)
com_rwm2_6 = next(com_generator_rwm2_6)
print('The number of communities in the sample : ' + str(len(com_rwm2_6)))

The number of communities in the sample : 98


The sample has a large variety of community structures. (Note: Giant Component = 107)